In [3]:
from wabs import *
import yaml

def yml(filepath='xpath.yml'):
    with open(filepath, 'r') as stream:
        try:
            parsed_yaml=yaml.safe_load(stream)
            return parsed_yaml
        except yaml.YAMLError as exc:
            print(exc)
            return 0

In [4]:
class Watem:
    
    def __init__(self, url=None):
        chrome_options = Options()
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.url = 'whatsapp.com' if url is None else url
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.w = WebDriverWait(self.driver, 10)
        print(self.driver.title)
    
    @property
    def get_driver(self):
        return self.driver

In [15]:
class W2(WhBase):
    
    def msg_xpath_by_text_last_index(self, text, get_text=False):
        txtm = "(//div[contains(@class,'_21Ahp') and contains(., '" + text + "')])[last()]"
        fxpt = txtm + "/ancestor::div[@data-testid='msg-container']"
        return fxpt if get_text==False else self.chat_msg(base=fxpt)
    
    def msg_xpath_by_text_time(self, text, time, get_text=False):
        txtm = "//div[contains(@class,'_21Ahp') and contains(., '" + text + "') and contains(.,'" + str(time) + "')]"
        fxpt = txtm + "/ancestor::div[@data-testid='msg-container']"
        return fxpt if get_text==False else self.chat_msg(base=fxpt)

    def msg_info_by_text_time(self, text, time):
        msg_xpath = self.msg_xpath_by_text_time(text, time)
        return self.chat_msg(base=msg_xpath)
        
    def msg_yield_get_by_sender(self, name=None, base="(//div[@data-testid='msg-container'])"):
        bi = base + "[contains(.,'" + name + "')]" if name is not None else base
        i, nonecount = 0, 0
        while nonecount<10:
            if i > 0: inx='last()-' + str(i)
            else: inx = 'last()'
            xpdic = self.chat_msg(base = bi + "[" + inx + "]")
            if xpdic is None: 
                nonecount = nonecount + 1
                self.just_move()
                tm.sleep(1)
            else: nonecount = 0
            yield xpdic, i, nonecount
            i = i + 1
    
    def pane_dyn_xpath(self, by_chatname=None, by_msgtxt=None, by_last_sender=None, get_text=False):
        contain = []
        if by_chatname is not None: 
            contain.append("contains(@title,'" + by_chatname + "')")
        if by_msgtxt is not None: 
            contain.append("contains(@data-testid='last-msg-status') and contains(.,'" + by_msgtxt + "')")
        if by_last_sender is not None: 
            contain.append("contains(@dir='auto') and contains(., '" + by_last_sender + "')")
        fxpt = "(//div[@id='pane-side'])//*[" + ' and '.join(map(str,contain)) + "]" + "//ancestor::div[@class='_8nE1Y']"
        rs = fxpt if get_text==False else self.side_pane(base=fxpt)
        return rs
    
    def general_inspection(self):
        dc = {
            'pn_search_textarea' : "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]",
            'pn_search' : "//div[@id='side']//div[@data-testid='chat-list-search']",
            'pn_search_filter' : "//div[@id='side']//button[@aria-label='Unread chats filter']",
            'chat_title' : "//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']",
            'chat_text_area' : "//footer[@class='_3E8Fg']//p",
            'msg_base' : "(//div[@id='main']//div[@data-testid='msg-container'])"
        }
    
    def ensure_click_attempt(self, hover_xpt, click_xpt, required_visibility_xpt):
        q = self.xpelem(hover_xpt)
        if q is not None:
            ActionChains(self.driver).move_to_element(q).pause(1).click(self.xpelem(click_xpt)).perform()
        else:
            self.just_move()
            tm.sleep(1)
            ActionChains(self.driver).move_to_element(self.xpelem(hover_xpt)).pause(.5).click(self.xpelem(click_xpt)).perform()
        p = self.xpelem(required_visibility_xpt)
        if p is not None: return p
        else: 
            print('ensure_click_attempt fail')
            self.reload
            return 0
            
    def action(self, chat_name='OPS & RO', search_text="battery backup degraded?", msg_time=None, forward=None,
               reply=None, react=None):
        x = self
        clk_1 = "//span[@data-testid='down-context']"
        clk_2="//ul//li[contains(.,'Forward message')]"
        clk_3= "//button[@title='Forward message']"
        select_chat_= lambda _chat_ : "(//div[@class='_8nE1Y' and contains(.,'" + _chat_ + "')])/" + \
        "ancestor::div[@class='lhggkp7q ln8gz9je rx9719la']"
        write_txt = "//div[@data-testid='chat-list-search']"
        clk_send_btn = "//span[@data-testid='send']"
        msg_write_area = "//footer[@class='_3E8Fg']//p"

        chat_selected, while_counter = False, 0
        a1 = x.pane_dyn_xpath(by_chatname=chat_name)
        if x.click(a1) == 0:
            tm.sleep(.5)
            if x.select_chat(chat_name) != 0:
                chat_selected = True
        else:
            chat_selected = True
        print('chat_selected; ', chat_selected)
        if chat_selected == True:
            if msg_time is not None: a2 = x.msg_xpath_by_text_time(search_text,msg_time)
            else: a2 = x.msg_xpath_by_text_last_index(search_text)
            print('Time is None, selecting by last index')
            move = 1 if x.just_move(a2) is None else 0
            while move == 0 or move > -5 and move !=1:
                while_counter= while_counter + 1
                x.just_move("(//div[@id='main']//div[@data-testid='msg-container'])[2]")
                if x.just_move(a2) == 0:
                    move = move - 1
                else:
                    move = 1
                    break
                print('while_counter:', while_counter,' inside while Move: ',move)
            print('move Outside: ', move)
            if move > 0:
                print('executute clicking operation')
                x.click("//span[@data-testid='down-context']")
                tm.sleep(.5)
                if self.xpelem("//ul//li") is None:
                    q = self.ensure_click_attempt(a2, a2 + "//span[@data-testid='down-context']", "//ul//li")
                    if q == 0:
                        print('msg ul/li not found.. run again')
                        return 0
                if forward is not None and len(forward)>1:
                    print('msg forward init')
                    x.click("//ul//li[contains(.,'Forward message')]")
                    x.click("//button[@title='Forward message']")
                    #x.move_click_send_text("//div[@data-testid='chat-list-search']","SM POOL")
                    x.paste_text("//div[@data-testid='chat-list-search']",forward)
                    if x.click(select_chat_(forward)) != 0:
                        tm.sleep(.5)
                        x.click("//span[@data-testid='send']")
                        return 1
                    else:
                        print('forward failed')
                elif reply is not None and len(reply) > 1:
                    x.click("//ul//li[contains(.,'Reply')]")
                    x.click_write_enter(msg_write_area, reply)
                    return 1
                elif react is not None:
                    x.click("//ul//li[contains(.,'React')]")
                    x.click("//div[@data-testid='reactions-option-0']")
                    return 1
            

In [16]:
x = W2()
x.reload
d = x.get_driver

WhatsApp
reload success, whatsapp ready


In [17]:
#x.action(chat_name='SOC SM', search_text="Network Update", msg_time=None, react='yes')

In [18]:
n = x.msg_yield_get_by_sender('Jalal Vaia')

In [19]:
n.__next__()

(None, 0, 1)

In [20]:
n.__next__()

(None, 1, 2)

In [21]:
n.__next__()

(None, 2, 3)

In [22]:
n.__next__()
n.__next__()

(None, 4, 5)

In [28]:
x.just_move("(//div[@data-testid='conversation-panel-body'])//span[contains(.,'YESTERDAY')]")

0

In [39]:


def action(d, k, v): 
    ac = ActionChains(d)
    if '_sendtxt_' in k: ac.move_to_element(v[0]).click().send_keys(v[1]).send_keys(Keys.RETURN).perform()
    elif '_clk_' in k: ac.click(xpelem(v[0])).perform()
    elif '_moveto_' in k: ac.move_to_element(xpelem(v[0])).perform()
    elif '_hoverclk_' in k: ac.move_to_element(xpelem(v[0])).click(xpelem(v[1])).perform()
    elif '_moveclk_' in k: ac.move_to_element(xpelem(v[0])).click().perform()
    else: return 0

In [40]:
select_chat_("SM POOL")

"(//div[@class='_8nE1Y' and contains(.,'SM POOL')])/ancestor::div[@class='lhggkp7q ln8gz9je rx9719la']"

In [47]:
def action(self, chat_name='OPS & RO', msg_text="battery backup degraded?", msg_time="6:33", forward='SM POOL',
          reply=None, delete=None):
    x = self
    clk_1 = "//span[@data-testid='down-context']"
    clk_2="//ul//li[contains(.,'Forward message')]"
    clk_3= "//button[@title='Forward message']"
    select_chat_= lambda _chat_ : "(//div[@class='_8nE1Y' and contains(.,'" + _chat_ + "')])/" + \
    "ancestor::div[@class='lhggkp7q ln8gz9je rx9719la']"
    write_txt = "//div[@data-testid='chat-list-search']"
    clk_send_btn = "//span[@data-testid='send']"
    
    msg_write_area = "//footer[@class='_3E8Fg']//p"

    chat_selected, while_counter = False, 0
    a1 = x.pane_dyn_xpath(by_chatname='OPS & RO')
    if x.click(a1) == 0:
        tm.sleep(.5)
        if x.select_chat('OPS & RO') != 0:
            chat_selected = True
    else:
        chat_selected = True
    print('chat_selected; ', chat_selected)
    if chat_selected == True:
        a2 = x.msg_xpath_by_text_time("battery backup degraded?","6:33")
        move = 1 if x.just_move(a2) is None else 0
        while move == 0 or move > -5 and move !=1:
            while_counter= while_counter + 1
            x.just_move("(//div[@id='main']//div[@data-testid='msg-container'])[2]")
            if x.just_move(a2) == 0:
                move = move - 1
            else:
                move = 1
                break
            print('while_counter:', while_counter,' inside while Move: ',move)
        print('move Outside: ', move)
        if move > 0:
            print('executute clicking operation')
            x.click("//span[@data-testid='down-context']")
            if forward is not None and len(forward)>1:
                print('msg forward init')
                x.click("//ul//li[contains(.,'Forward message')]")
                x.click("//button[@title='Forward message']")
                #x.move_click_send_text("//div[@data-testid='chat-list-search']","SM POOL")
                x.paste_text("//div[@data-testid='chat-list-search']","SM POOL")
                if x.click(select_chat_("SM POOL")) != 0:
                    tm.sleep(.5)
                    x.click("//span[@data-testid='send']")
                else:
                    print('forward failed')
            elif reply is not None and len(reply) > 1:
                x.click("//ul//li[contains(.,'Reply')]")
                x.click_write_enter(msg_write_area, reply)
            elif react is not None:
                x.click("//ul//li[contains(.,'React')]")
                x.click("div[@data-testid='reactions-option-0']")
        

chat_selected;  True
move Outside:  1
executute clicking operation


In [48]:
x.click("//div[@data-testid='content']")

1

In [15]:
x.click("(//div[@class='_8nE1Y' and contains(.,'Jalal Vaia')])/ancestor::div[@class='lhggkp7q ln8gz9je rx9719la']")

1

In [21]:
x.click("(//div[@class='_8nE1Y' and contains(.,'SM POOL')])/ancestor::div[@class='lhggkp7q ln8gz9je rx9719la']")

1

In [73]:
def log(xpt, rs):
    tm = dt.datetime.now().strftime('%H:%M %Y-%m-%d')
    tx = "present" if rs is not None else "NF"
    fp = open('commands_output.txt', 'a+')
    fp.write(xpt + ',' + tx + ',' + tm)
    fp.close()


def xpe(d, xpt, wait=5):
    w = WebDriverWait(d, wait)
    try: return w.until(EC.presence_of_element_located((By.XPATH, xpt))).text
    except: return None

def itr(d, xpt):
    if type(xpt) is str: return xpe(d, xpt)
    elif type(xpt) is list: return [xpe(n) for n in xpt]
    elif type(xpt) is dict:
        return {k : [xpe(d, n) for n in v] if type(v) is list else xpe(d, v) for k, v in xpt.items()}
    else:
        print('xpt is not str or list or dict')
        return xpt

In [80]:
obj = W2()
d = obj.get_driver

(9) WhatsApp


In [ ]:
itr(d, )

In [ ]:
dc = {'sender' : [sender],
      'text' : [self.xptext(bi + "//div[@class='_21Ahp']/span[1]/span")],
      'q_sender' : [self.xptext(bi + "//div[@class='_3pMOs yKTUI']//div[1]/span")],
      'q_text' : [self.xptext(bi + "//div[@class='_3pMOs yKTUI']//div[2]/span")],
      'datetime' : [dttm]}

In [81]:
wa_home = init_info = "//div[@data-testid='intro-text'][contains(.,'Use WhatsApp on up to 4 linked devices and 1 phone at the same time.')]"

pnbs = "(//div[@id='pane-side']//div[@class='_8nE1Y'])"
pn_search_textarea = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
pn_search = "//div[@id='side']//div[@data-testid='chat-list-search']"
pn_search_back ="//div[@id='side']//span[@data-testid='search']"
#pn_search_cancel = "//div[@id='side']//span[contains(@data-testid,'x-alt')]"
pn_search_filter = "//div[@id='side']//button[@aria-label='Unread chats filter']"
pn_chat_name = lambda i : pnbs + '[' + str(i) + ']' + "//div[@class='_21S-L']//span[@dir='auto']"
pn_last_text = lambda i : pnbs + '[' + str(i) + ']' + "//span[@data-testid='last-msg-status']"
pn_last_sender = lambda i : pnbs + '[' + str(i) + ']' + "//div[@class='vQ0w7']//span[@dir='auto']"
pn_last_msg_time = lambda i : pnbs + '[' + str(i) + ']' + "/div[@class='y_sn4']/div[@class='Dvjym']"
pn_msg_unread_count = lambda i : pnbs + '[' + str(i) + ']' + "//div[@class='Dvjym']//span[@data-testid='icon-unread-count']"
pnbs_by_chatname = lambda arg : "//div[@class='_21S-L']//span[@dir='auto' and contains(.,'" + arg + "')]//ancestor::div[@class='_8nE1Y']"
pnbs_by_last_text = ""
pnbs_by_last_text_sender = ""
pnbs_by_time = ""
pn_unread_msg_by_chatname = lambda chatname: "//div[@class='_21S-L']//span[@dir='auto' and contains(text(),'" + chatname + "']//ancestor::div[@class='_8nE1Y']//div[@class='Dvjym']//span[@data-testid='icon-unread-count']"

In [105]:

pn_search_textarea = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
pn_search_filter = "//div[@id='side']//button[@aria-label='Unread chats filter']"
            

In [106]:
#(//div[contains(@class,'_27K43 _31p5Q') or contains(@class, 'ItfyB _3nbHh _1mRgz') or contains(@class, 'ItfyB _3nbHh')])[contains(.,'Jalal Vai')]

In [107]:
msg_bysender('jalal Vai')

"(//div[@data-testid='msg-container']/div)[contains(.,'jalal Vai')][last()-3]"

In [111]:
chat_title = "//div[@id='main']//span[@data-testid='conversation-info-header-chat-title']"
msg_menu = "//div[@class='_2sDI2']/ul//li"

def pane_chat_by_name(chatname):
    a1 = "//div[@class='_21S-L']//span[@dir='auto' and contains(.,'" + chatname + "')]"
    a2 = "//ancestor::div[@class='_8nE1Y']"
    dc = {
        'chatdiv' : a1 + a2,
        'chatname': a1,
        'unread_msg' : a1 + a2 + "//div[@class='Dvjym']//span[@data-testid='icon-unread-count']",
        'last_text' : a1 + a2 + "//span[@data-testid='last-msg-status']",
        'last_sender' : a1 + a2 + "//div[@class='vQ0w7']//span[@dir='auto']",
        'last_msg_time' : a1 + a2 + "/div[@class='y_sn4']/div[@class='Dvjym']"
    }
    return dc

def pane_chat_by_text(text): "//span[@data-testid='last-msg-status' and contains(.,'" + text + "')]"
    a1 = "//span[@data-testid='last-msg-status' and contains(.,'" + text + "')]"
    a2 = "//ancestor::" + base
    dc = {
        'chatdiv' : a1 + a2,
        'chatname': a1,
        'unread_msg' : a1 + a2 + "//div[@class='Dvjym']//span[@data-testid='icon-unread-count']",
        'last_text' : a1 + a2 + "//span[@data-testid='last-msg-status']",
        'last_sender' : a1 + a2 + "//div[@class='vQ0w7']//span[@dir='auto']",
        'last_msg_time' : a1 + a2 + "/div[@class='y_sn4']/div[@class='Dvjym']"
    }
    print(dc)
    return dc

SyntaxError: invalid syntax (3498806707.py, line 18)

In [92]:
itr(d, pane_chat_by_text('Update'))

{'chatdiv': "//span[@data-testid='last-msg-status' and contains(.,'Update')]//ancestor::div[@class='_8nE1Y']", 'chatname': "//span[@data-testid='last-msg-status' and contains(.,'Update')]", 'unread_msg': "//span[@data-testid='last-msg-status' and contains(.,'Update')]//ancestor::div[@class='_8nE1Y']//div[@class='Dvjym']//span[@data-testid='icon-unread-count']", 'last_text': "//span[@data-testid='last-msg-status' and contains(.,'Update')]//ancestor::div[@class='_8nE1Y']//span[@data-testid='last-msg-status']", 'last_sender': "//span[@data-testid='last-msg-status' and contains(.,'Update')]//ancestor::div[@class='_8nE1Y']//div[@class='vQ0w7']//span[@dir='auto']", 'last_msg_time': "//span[@data-testid='last-msg-status' and contains(.,'Update')]//ancestor::div[@class='_8nE1Y']/div[@class='y_sn4']/div[@class='Dvjym']"}


{'chatdiv': 'Network Operations\n6:06 pm\nMahafuj\n: \nUpdate of Site Down at 18:00 On 02-02-2023 Region: 2G/3G/4G National: 40/23/49 DHK_M: 7/8/9 DHK_N: 1/1/2 DHK_S: 0/0/1 DHK_O: 2/2/2 CTG_M: 2/2/4 CTG_N: 3/0/3 CTG_S: 7/1/7 COM: 3/3/3 NOA: 0/0/0 SYL: 2/0/2 MYM: 4/1/5 BAR: 3/2/4 KHL: 1/2/1 KUS: 1/1/1 RAJ: 3/0/3 RANG: 1/0/2\n13',
 'chatname': 'Mahafuj\n: \nUpdate of Site Down at 18:00 On 02-02-2023 Region: 2G/3G/4G National: 40/23/49 DHK_M: 7/8/9 DHK_N: 1/1/2 DHK_S: 0/0/1 DHK_O: 2/2/2 CTG_M: 2/2/4 CTG_N: 3/0/3 CTG_S: 7/1/7 COM: 3/3/3 NOA: 0/0/0 SYL: 2/0/2 MYM: 4/1/5 BAR: 3/2/4 KHL: 1/2/1 KUS: 1/1/1 RAJ: 3/0/3 RANG: 1/0/2',
 'unread_msg': '13',
 'last_text': 'Mahafuj\n: \nUpdate of Site Down at 18:00 On 02-02-2023 Region: 2G/3G/4G National: 40/23/49 DHK_M: 7/8/9 DHK_N: 1/1/2 DHK_S: 0/0/1 DHK_O: 2/2/2 CTG_M: 2/2/4 CTG_N: 3/0/3 CTG_S: 7/1/7 COM: 3/3/3 NOA: 0/0/0 SYL: 2/0/2 MYM: 4/1/5 BAR: 3/2/4 KHL: 1/2/1 KUS: 1/1/1 RAJ: 3/0/3 RANG: 1/0/2',
 'last_sender': 'Mahafuj',
 'last_msg_time': '6:0

(14) WhatsApp


In [49]:
//div[@class='_2KKXC']

"//div[@class='_21S-L']//span[@dir='auto' and contains(.,'Halim Vai')]//ancestor::div[@class='_8nE1Y']"

In [60]:
z = xpe(d, pn_unread_msg_by_chatname('SOC-RAN Ops Collaboration')).text
z

AttributeError: 'NoneType' object has no attribute 'text'

In [57]:

pane_select_chat_ = lambda _chat_: "//div[@class='_21S-L']//span[@dir='auto' and contains(.,'" + _chat_ + \
"')]//ancestor::div[@class='_8nE1Y']"

getmsg_txt_ = lambda _txt_ : "(//div[@class='_21Ahp'])[contains(.,'" + _txt_ + "')]" + \
"[last()]/ancestor::div[@data-testid='msg-container'])[last()]"

clk_1 = "//span[@data-testid='down-context']"
clk_2="//ul//li[contains(.,'Forward message')]",
clk_3= "//button[@title='Forward message']"
select_chat_= lambda _chat_ : "(//div[@class='_8nE1Y' and contains(.,'" + _chat_ + "')])/" + \
"ancestor::div[@class='lhggkp7q ln8gz9je rx9719la']//div[@data-testid='visual-checkbox']"
write_txt = "//div[@data-testid='chat-list-search']"
clk_send_btn = "//span[@data-testid='send']"

In [ ]:

chat_search = "//div[@id='main']//span[@data-testid='search-alt']"
chat_search_btn = "//div[@id='main']//div[@class='_3ndVb']"
chat_text_area = "//footer[@class='_3E8Fg']//p"
msg_base = "(//div[@id='main']//div[@data-testid='msg-container'])"
mbs1 = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]"
mbs2 = "(//div[@id='main']//div[@class='ItfyB _3nbHh'])[last()]"
msg_arrow = "(//div[@data-testid='msg-container'])[last()]//span[@data-testid='down-context']"
msg_menu_select = lambda bytext : "//ul//li[contains(.,'" + bytext + "')]"

In [ ]:
wh = W2()
ws = wh.reload
d = wh.get_driver



#div[@data-testid="chat-list"]
#    - aria-label="Chat list", aria-rowcount= number
#data-testid="lock-small-v2
#data-testid="conversation-header"
#data-testid="conversation-panel-body"
#    - "div[@data-testid='conversation-panel-messages']", "div[@class = '_5kRIK']"
#    - div[@class='_35lRO']  
#data-testid="conversation-info-header"
#data-testid="chat-subtitle"
#msg_nav_arrow = "div[@class='pboYv']/span"
#id="hard_expire_time"
#data-time="1690306368.855"
#unix_time = '1690306368.855"
#data-testid="drawer-middle"
#data-testid="drawer-left"

In [ ]:
webElement.isEnabled()
webElement.isDisplayed()

In [5]:
# if any string enclosed with understore menning it need to replaced with desirable. _x_ - means _x_ is in value as string so replace with your desire
#if _and_ then generally fuction call  that return 2 value
# istrue = check true
# isfalse = check false
# string contains ||| &&& have to splited for checking condition
# xFC: name of function, arg, kwarg
# xMT: class object, method name, arg, kwargs
#if _name_ in key then replace _name_ with desire for every value
#if _i_ in key then replace _i_ with index for every value
#if _txt_ in key then replace _txt_ with desired for value
#if clk in key then perform clk operation after above 3
#if sendenter in then first send_keys and finally enter
#if sendkey in key then perform send_keys operation
#if sendtxt in key then perform send_keys operation
#if move in key then perform move_to_element operation
#if info in key then element.text operation
#if break in key and value is true then loop break
#if nothing match then get webelement and return